# Music Recommendation System using Spotify API

![](https://www.protocol.com/media-library/spotify-app-with-headphones-on.png?id=29171621&width=1000&height=750&quality=85&coordinates=0%2C0%2C0%2C0)

## Recommendation systems
**Recommendation systems** are algorithms and techniques designed to provide personalized suggestions or recommendations to users. These systems analyze user data and item attributes to generate relevant and tailored recommendations. Recommendation systems are widely used in various applications such as e-commerce (product recommendations), streaming services (movie/music recommendations), news websites (article recommendations), and more to enhance user experiences and drive engagement. 

In this notebook, I will be demonstrating one such application of Recommendation systems by building a Music Recommendation System using the Spotify API. The music recommendation system will assist users in discovering new and relevant musical content based on their preferences and listening behaviour.

### Table of Contents
1. Introduction
2. Spotify API

## Spotify API
**Spotify API** (Application Programming Interface) is a set of web services and tools provided by Spotify that allows developers to interact with the Spotify platform and access various music-related data and functionalities. Developers can use the Spotify API to integrate Spotify's vast music catalog, user playlists, and user-related data into their own applications, websites, or services.

I will be using the Spotify API for requesting and collecting real-time music data from Spotify. For accessing the API you first have to sign up for a developer account on Spotify. As I have already done this I am skipping this step. For more information you can check out the [Spotify API documentation](https://developer.spotify.com/documentation/web-api).

To get started with building a Music Recommendation System, we first need to have an access token. The access token serves as a temporary authorization credential, allowing the code to make authenticated requests to the Spotify API on behalf of the application. The access token can be requested in the following way

In [2]:
client_creds_enc.decode()

'Y2U0NTM1ZjNlMmI1NDcyYzgxZGFkYzExMjA3ZTNkMTI6NTc5OWRjMTk1N2ExNDdmNDg1ZmUyOTQwYjI4Yzc0MTQ='

In [3]:
client_creds_enc

b'Y2U0NTM1ZjNlMmI1NDcyYzgxZGFkYzExMjA3ZTNkMTI6NTc5OWRjMTk1N2ExNDdmNDg1ZmUyOTQwYjI4Yzc0MTQ='

In [5]:
# Importing required libraries
import requests
import base64
import pandas as pd
from IPython.display import Image

In [148]:
# Replace with your Client ID and Client Secret
CLIENT_ID = 'ce4535f3e2b5472c81dadc11207e3d12'
CLIENT_SECRET = '5799dc1957a147f485fe2940b28c7414'

# Encode the client ID and client secret to base64
client_creds = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_creds_enc = base64.b64encode(client_creds.encode())

# Request the access token
TOKEN_URL = "https://accounts.spotify.com/api/token"
headers = {
    'Authorization': f'Basic {client_creds_enc.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(TOKEN_URL, data=data, headers=headers)

# Check if access token was obtained or not
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


With the access token obtained, we can now make authorized requests to Spotify and retrieve music data, such as tracks, albums, artists, and user information, which is fundamental for our Music Recommendation System. Now to collect music data from Spotify we need to first install the [Spotipy](https://spotipy.readthedocs.io/en/2.22.1/) library, a Python library that provides access to Spotify’s web API

In [4]:
# Install the spotipy library
# !pip install spotipy

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Set up Spotipy with the access token
sp = spotipy.Spotify(auth=access_token)

In [153]:
# Get info about an artist
def get_artist_info(artist_id):
    artist = sp.artist(artist_id=artist_id)

    num_followers = artist['followers']['total']
    genres = artist['genres']
    artist_id = artist['id']
    name = artist['name']
    popularity_score = artist['popularity']
    img_url = artist['images'][2]['url']

    print(name)
    display(Image(url=img_url))
    print("Total followers:", num_followers)
    print("Popularity:", popularity_score)
    print("Genres:", *genres)

In [154]:
get_artist_info(artist_id=artist_id)

Taylor Swift


Total followers: 85379848
Popularity: 100
Genres: pop


In [168]:
# Get info on artist's albums
def get_artist_albums(artist_id):
    artist_name = sp.artist(artist_id=artist_id)['name']
    artist_album = sp.artist_albums(artist_id=artist_id, album_type=['album', 'single'], limit=50)
    total_albums = artist_album['total']

    albums = []
    for i in range(total_albums):
        album = artist_album['items'][i]
        album_id = album['id']
        album_name = album['name']
        release_date = album['release_date']
        total_tracks = album['total_tracks']

        album_info = {'album_id': album_id,
                      'name': album_name,
                      'released': release_date,
                      'total_tracks': total_tracks}
        albums.append(album_info)
    
    print(f"Albums from {artist_name}")
    albums_df = pd.DataFrame(albums)
    return albums_df

In [169]:
get_artist_albums(artist_id=artist_id)

Albums from Taylor Swift


,album_id,name,released,total_tracks
0,5AEDGbliTTfjOB8TSm1sxt,Speak Now (Taylor's Version),2023-07-07,22
1,1fnJ7k0bllNfL1kVdNVW1A,Midnights (The Til Dawn Edition),2023-05-26,23
2,3lS1y25WAhcqJDATJK70Mq,Midnights (3am Edition),2022-10-22,20
3,151w1FgRZfnKZA9FEcg9Z3,Midnights,2022-10-21,13
4,6kZ42qRrzov54LcAk4onW9,Red (Taylor's Version),2021-11-12,30
5,4hDok0OAJd57SGIT8xuWJH,Fearless (Taylor's Version),2021-04-09,26
6,6AORtDjduMM3bupSWzbTSG,evermore (deluxe version),2021-01-07,17
7,2Xoteh7uEpea4TohMxjtaq,evermore,2020-12-11,15
8,0PZ7lAru5FDFHuirTkWe9Z,folklore: the long pond studio sessions (from ...,2020-11-25,34
9,1pzvBxYgT6OVwJLtHkrdQK,folklore (deluxe version),2020-08-18,17


In [150]:
# Get artist's top 10 tracks
def get_artist_tracks(artist_id):
    artist_name = sp.artist(artist_id=artist_id)['name']
    artist_top_tracks = sp.artist_top_tracks(artist_id=artist_id)
    
    top_tracks = []
    for i in range(10):
        track = artist_top_tracks['tracks'][i]
        duration = track['duration_ms']
        track_id = track['id']
        track_name = track['name']
        popularity = track['popularity']
        album = track['album']['name']

        track_info = {'track_id': track_id,
                      'name': track_name,
                      'duration(ms)': duration,
                      'album': album,
                      'popularity_score': popularity}
        top_tracks.append(track_info)

    print(f"Top 10 tracks from {artist_name}")
    tracks_df = pd.DataFrame(top_tracks)
    return tracks_df

In [151]:
# Extract Taylor Swift's top 10 tracks
get_artist_tracks(artist_id=artist_id)

Top 10 tracks from Taylor Swift


,track_id,name,duration(ms),album,popularity_score
0,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,178426,Lover,100
1,0V3wPSX9ygBnCm8psDIegu,Anti-Hero,200690,Midnights,93
2,1p80LdxRV74UKvL8gnD7ky,Blank Space,231826,1989,83
3,4lIxdJw6W3Fg4vUIYCB0S5,Style,231000,1989,83
4,3hUxzQpSfdDqwM3ZTFQY0K,august,261922,folklore,93
5,4R2kfaDFhslZEMJqAFNpdd,cardigan,239560,folklore,91
6,7KokYm8cMIXCsGVmUvKtqf,Karma,204852,Midnights,90
7,1R0a2iXumgCiFb7HEZ7gUE,Don’t Blame Me,236413,reputation,91
8,1dGr1c8CrMLDpV6mPbImSI,Lover,221306,Lover,91
9,5kHMfzgLZP95O9NBy0ku4v,I Can See You (Taylor’s Version) (From The Vault),273186,Speak Now (Taylor's Version),90


In [187]:
audio=sp.audio_features("1BxfuPKGuaTgP7aM0Bbdwr")
audio

[{'danceability': 0.552,
  'energy': 0.702,
  'key': 9,
  'loudness': -5.707,
  'mode': 1,
  'speechiness': 0.157,
  'acousticness': 0.117,
  'instrumentalness': 2.06e-05,
  'liveness': 0.105,
  'valence': 0.564,
  'tempo': 169.994,
  'type': 'audio_features',
  'id': '1BxfuPKGuaTgP7aM0Bbdwr',
  'uri': 'spotify:track:1BxfuPKGuaTgP7aM0Bbdwr',
  'track_href': 'https://api.spotify.com/v1/tracks/1BxfuPKGuaTgP7aM0Bbdwr',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1BxfuPKGuaTgP7aM0Bbdwr',
  'duration_ms': 178427,
  'time_signature': 4}]